In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[11],9
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004}"
nv,-,1000
tsfc,300,300


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.135159                    1  0.284166
0.000750        2  0.010779                    2  0.093240
0.001052        3  0.008704                    3  0.050748
0.001476        4  0.003673                    4  0.010712
0.002070        5 -0.001149                    5 -0.016951
0.002904        6 -0.005399                    6 -0.034370
0.004074        7 -0.009281                    7 -0.039931
0.005714        8 -0.013051                    8 -0.041487
0.008015        9 -0.016945                    9 -0.042303
0.011243       10 -0.021130                   10 -0.042164
0.015771       11 -0.025744                   11 -0.043766
0.022122       12 -0.031029                   12 -0.045828
0.031031       13 -0.038005                   13 -0.051282
0.043528       14 -0.050464                   14 -0.062966
0.061057       15 -0.058837                   15 -0.068559
0.085645       16 -0.056266                   16 -0.058931
0.120136       17 -0.049540                   17 -0.040869
0.168516       18 -0.040602                   18 -0.021155
0.236378       19 -0.024440                   19 -0.001986
0.331549       20  0.004690                   20  0.017133
0.465100       21  0.063415                   21  0.053765
0.652400       22  0.181212                   22  0.151861
0.915100       23  0.211797                   23  0.176156
1.283650       24  0.150577                   24  0.138536
1.800600       25  0.104316                   25  0.118044
2.525700       26  0.061807                   26  0.086370
3.542800       27  0.036705                   27  0.057191
4.969550       28  0.021989                   28  0.029249
6.970850       29  0.012397                   29  0.009622
9.778100       30  0.006108                   30  0.003553
13.715850      31  0.002197                   31  0.004676
19.239350      32  0.000065                   32  0.003570
26.987250      33 -0.000769                   33  0.000951
37.855300      34 -0.000773                   34 -0.000749
53.100050      35 -0.001389                   35 -0.001829
73.887500      36 -0.001965                   36 -0.002283
97.662500      37 -0.002285                   37 -0.002579
121.437500     38 -0.002522                   38 -0.002806
145.212500     39 -0.002494                   39 -0.002750
168.987500     40 -0.002667                   40 -0.003068
192.762500     41 -0.002831                   41 -0.003338
216.537500     42 -0.002984                   42 -0.003623
240.312500     43 -0.003087                   43 -0.003840
264.087500     44 -0.003135                   44 -0.004068
287.862500     45 -0.003119                   45 -0.004163
311.637500     46 -0.003024                   46 -0.003971
335.412500     47 -0.002839                   47 -0.003729
359.187500     48 -0.002565                   48 -0.003487
382.962500     49 -0.002165                   49 -0.003472
406.737500     50 -0.001548                   50 -0.003309
430.512500     51 -0.000720                   51 -0.002696
454.287500     52  0.000331                   52 -0.001687
478.062500     53  0.001612                   53  0.000216
501.837500     54  0.003127                   54  0.002171
525.612500     55  0.004893                   55  0.004961
549.387500     56  0.006905                   56  0.007354
573.162500     57  0.009191                   57  0.010040
596.937500     58  0.011814                   58  0.012264
620.712500     59  0.014860                   59  0.014399
644.487500     60  0.018542                   60  0.016790
668.262500     61  0.022985                   61  0.019813
692.037500     62  0.027717                   62  0.023348
715.812500     63  0.031291                   63  0.025727
739.587500     64  0.033872                   64  0.027854
763.362500     65  0.033154                   65  0.027195
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -4.705975  0.000000 -4.705975 -4.592388  7.849115e-09   
0.000624    2     -4.705980  0.000014 -4.705965 -4.592395  2.821201e-05   
0.000876    3     -4.705982  0.000017 -4.705965 -4.592398  3.422928e-05   
0.001229    4     -4.705985  0.000020 -4.705964 -4.592403  4.057244e-05   
0.001723    5     -4.705988  0.000024 -4.705964 -4.592408  4.677712e-05   
0.002417    6     -4.705993  0.000029 -4.705964 -4.592416  5.271356e-05   
0.003391    7     -4.705999  0.000034 -4.705965 -4.592425  5.861470e-05   
0.004757    8     -4.706008  0.000041 -4.705966 -4.592439  6.538248e-05   
0.006672    9     -4.706020  0.000050 -4.705970 -4.592456  7.385519e-05   
0.009359    10    -4.706036  0.000061 -4.705975 -4.592481  8.497462e-05   
0.013128    11    -4.706058  0.000074 -4.705984 -4.592515  1.001102e-04   
0.018415    12    -4.706090  0.000090 -4.706000 -4.592563  1.206952e-04   
0.025830    13    -4.706136  0.000108 -4.706028 -4.592632  1.490528e-04   
0.036232    14    -4.706205  0.000130 -4.706075 -4.592733  1.874006e-04   
0.050823    15    -4.706319  0.000158 -4.706162 -4.592895  2.403704e-04   
0.071291    16    -4.706514  0.000210 -4.706304 -4.593154  3.325734e-04   
0.100000    17    -4.706825  0.000329 -4.706496 -4.593542  5.206090e-04   
0.140271    18    -4.707301  0.000569 -4.706732 -4.594104  8.871094e-04   
0.196760    19    -4.708017  0.001013 -4.707004 -4.594905  1.546837e-03   
0.275997    20    -4.709059  0.001826 -4.707233 -4.596029  2.652886e-03   
0.387100    21    -4.710485  0.003313 -4.707171 -4.597545  4.393867e-03   
0.543100    22    -4.712031  0.006031 -4.706000 -4.599226  7.068436e-03   
0.761700    23    -4.711969  0.010662 -4.701307 -4.599220  1.099562e-02   
1.068500    24    -4.709403  0.015793 -4.693610 -4.596701  1.487901e-02   
1.498800    25    -4.705504  0.019570 -4.685935 -4.592934  1.817440e-02   
2.102400    26    -4.700696  0.022220 -4.678476 -4.588269  2.195042e-02   
2.949000    27    -4.695833  0.023556 -4.672278 -4.583412  2.575576e-02   
4.136600    28    -4.691184  0.024070 -4.667114 -4.578504  2.889406e-02   
5.802500    29    -4.686774  0.023999 -4.662775 -4.573598  2.976056e-02   
8.139200    30    -4.682657  0.023313 -4.659343 -4.568705  2.753182e-02   
11.417000   31    -4.678913  0.021942 -4.656971 -4.564019  2.422565e-02   
16.014700   32    -4.675650  0.019876 -4.655775 -4.559743  2.249637e-02   
22.464000   33    -4.672990  0.017266 -4.655725 -4.556076  2.155681e-02   
31.510500   34    -4.671022  0.014473 -4.656548 -4.553030  1.952996e-02   
44.200100   35    -4.669551  0.011840 -4.657711 -4.550250  1.562375e-02   
62.000000   36    -4.669585  0.008946 -4.660639 -4.548988  1.050489e-02   
85.775000   37    -4.672487  0.006314 -4.666174 -4.551508  6.594659e-03   
109.550000  38    -4.677700  0.005092 -4.672608 -4.557402  5.223356e-03   
133.325000  39    -4.685295  0.005586 -4.679710 -4.566451  6.367876e-03   
157.100000  40    -4.694800  0.008066 -4.686734 -4.577650  9.820043e-03   
180.875000  41    -4.706323  0.012077 -4.694246 -4.591062  1.459301e-02   
204.650000  42    -4.720047  0.017828 -4.702219 -4.606799  2.092891e-02   
228.425000  43    -4.736424  0.025800 -4.710623 -4.625332  2.925661e-02   
252.200000  44    -4.755864  0.036547 -4.719317 -4.646977  4.008582e-02   
275.975000  45    -4.778882  0.050737 -4.728145 -4.672241  5.389224e-02   
299.750000  46    -4.806089  0.069160 -4.736930 -4.701702  7.162821e-02   
323.525000  47    -4.838180  0.092733 -4.745447 -4.735893  9.463490e-02   
347.300000  48    -4.875931  0.122488 -4.753443 -4.775793  1.240328e-01   
371.075000  49    -4.920028  0.159362 -4.760666 -4.821953  1.603701e-01   
394.850000  50    -4.970166  0.203403 -4.766763 -4.874103  2.027406e-01   
418.625000  51    -5.026751  0.255628 -4.771123 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')